In [1]:
import os

#### checking the current root path

In [2]:
%pwd

'c:\\Users\\Hp\\Documents\\GitHub\\coccidiosis-disease-classification\\notebooks'

#### changing my root path

In [3]:
os.chdir("../")

%pwd

'c:\\Users\\Hp\\Documents\\GitHub\\coccidiosis-disease-classification'

# Entity

### update the entity (written type of the func.)

creating the data ingestion config

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Configuration

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# Component

In [7]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info(f"{filename} download! with following info: \n{headers}")
                
                # check if file is not empty
                if os.path.getsize(self.config.local_data_file) == 0:
                    raise Exception(f"downloaded file is empty")
            except Exception as e:
                logger.error(f"error downloading the file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))} bytes")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        # check file size before extraction
        if os.path.getsize(self.config.local_data_file) > 0:
            try:
                with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                    zip_ref.extractall(unzip_path)
                logger.info(f"file extracted successfully to {unzip_path}")
            except zipfile.BadZipFile:
                logger.error(f"corrupted zip file: {self.config.local_data_file}")
                raise
            except Exception as e:
                logger.error(f"error during extraction: {e}")
                raise
        else:
            logger.error("file is empty or not downloaded properly")
            raise Exception("invalid zip file: file is empty or corrupted.")

# Pipeline

### Creating Pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"an error occured: {e}")
    raise

[2024-10-23 22:30:30,550: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-23 22:30:30,553: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-23 22:30:30,555: INFO: common: created directory at: artifacts]
[2024-10-23 22:30:30,558: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-23 22:30:39,161: INFO: 4237178886: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1B7C:1E7709:1736:1D55:671921DF
Accept-Ranges: bytes
Date: Wed, 23 Oct 2024 17:00:32 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10221-MAA
X-Cache: HIT
